**NLP**

*CC6205-1 - Otoño 2022*

In [1]:
# !pip install --upgrade pandas

import pandas as pd
import numpy as np
# pd.set_option("max_rows", None)
import pickle
from string import punctuation
import re

from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
# from astropy.visualization import hist
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
import random

import plotly.express as px
import time
import datetime

from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support

import os
import shutil

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report, accuracy_score, roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
sentiments = "anger fear joy sadness".split()
intensities = "low medium high".split()

In [3]:
df_train = pickle.load(open("df_train.pickle", "rb"))

In [4]:
df_representation = pickle.load(open("df_representation_v1.pickle", "rb"))
df_representation_v2 = pickle.load(open("df_representation_v2.pickle", "rb"))
df_representation_v3 = pickle.load(open("df_representation_v3.pickle", "rb"))
df_representation_v4 = pickle.load(open("df_representation_v4.pickle", "rb"))

In [18]:
CLF = {sen: None for sen in sentiments}

sen = "anger"
indexs = df_train[df_train["sen"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v3.pickle", "rb"))

X = df_representation_v3.loc[indexs][cols_selected_sen]
y = df_train.loc[X.index]["int"].replace({"low": 0, "medium": 1, "high": 2})

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', class_weight="balanced", probability=True))
clf.fit(X, y)
CLF[sen] = clf
print(sen, clf.score(X, y))

sen = "joy"
indexs = df_train[df_train["sen"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v3.pickle", "rb"))

X = df_representation_v3.loc[indexs][cols_selected_sen]
y = df_train.loc[X.index]["int"].replace({"low": 0, "medium": 1, "high": 2})

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', class_weight="balanced", probability=True))
clf.fit(X, y)
CLF[sen] = clf
print(sen, clf.score(X, y))

sen = "fear"
indexs = df_train[df_train["sen"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v4.pickle", "rb"))

X = df_representation_v4.loc[indexs][cols_selected_sen]
y = df_train.loc[X.index]["int"].replace({"low": 0, "medium": 1, "high": 2})

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', class_weight="balanced", probability=True))
clf.fit(X, y)
CLF[sen] = clf
print(sen, clf.score(X, y))

sen = "sadness"
indexs = df_train[df_train["sen"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v2.pickle", "rb"))

X = df_representation_v2.loc[indexs][cols_selected_sen]
y = df_train.loc[X.index]["int"].replace({"low": 0, "medium": 1, "high": 2})

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', class_weight="balanced", probability=True))
clf.fit(X, y)
CLF[sen] = clf
print(sen, clf.score(X, y))

anger 0.8852284803400637
joy 0.8082039911308204
fear 0.7684964200477327
sadness 0.9011627906976745


In [6]:
# Datasets que deberán predecir para la competencia.
target = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/anger-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/fear-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/joy-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/sadness-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE'])
}

In [19]:
df_test = pd.DataFrame()
for sen in sentiments:
    df_test = pd.concat([df_test, target[sen]], axis=0)
df_test = df_test.set_index("id")

In [16]:
df_representation_test_v1 = pickle.load(open("df_representation_test_v1.pickle", "rb"))
df_representation_test_v2 = pickle.load(open("df_representation_test_v2.pickle", "rb"))
df_representation_test_v3 = pickle.load(open("df_representation_test_v3.pickle", "rb"))
df_representation_test_v4 = pickle.load(open("df_representation_test_v4.pickle", "rb"))

In [39]:
relevant_cols = set()
for sen in sentiments:
    for i in range(1, 4+1):
        cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v{i}.pickle", "rb"))
        relevant_cols = relevant_cols.union(set(cols_selected_sen))

In [42]:
drop_cols_v1 = set(df_representation_test_v1.columns).difference(set(df_representation.columns))
add_cols_v1 = set(df_representation.columns).difference(set(df_representation_test_v1.columns)).intersection(relevant_cols)
new_df_representation_test_v1 = df_representation_test_v1.drop(columns=drop_cols_v1)
d = dict.fromkeys(add_cols_v1, 0)
new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

...

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v1 = new_df_representation_test_v1.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/675109465.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [48]:
new_df_representation_test_v2 = df_representation_test_v2

In [47]:
drop_cols_v3 = set(df_representation_test_v3.columns).difference(set(df_representation_v3.columns))
add_cols_v3 = set(df_representation_v3.columns).difference(set(df_representation_test_v3.columns)).intersection(relevant_cols)
new_df_representation_test_v3 = df_representation_test_v3.drop(columns=drop_cols_v3)
d = dict.fromkeys(add_cols_v3, 0)
new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

...

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v3 = new_df_representation_test_v3.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/8751822.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

In [49]:
drop_cols_v4 = set(df_representation_test_v4.columns).difference(set(df_representation_v4.columns))
add_cols_v4 = set(df_representation_v4.columns).difference(set(df_representation_test_v4.columns)).intersection(relevant_cols)
new_df_representation_test_v4 = df_representation_test_v4.drop(columns=drop_cols_v4)
d = dict.fromkeys(add_cols_v4, 0)
new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

...

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df_representation_test_v4 = new_df_representation_test_v4.assign(**d)
C:\Users\felip\AppData\Local\Temp/ipykernel_30368/2309403582.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

In [59]:
def predict_target(dataset, classifier, labels):
    # Predecir las probabilidades de intensidad de cada elemento del target set.
    predicted = pd.DataFrame(classifier.predict_proba(dataset), columns=labels)
    # Agregar ids
    predicted['id'] = dataset.index
    # Reordenar las columnas
    predicted = predicted[['id', 'low', 'medium', 'high']]
    return predicted

In [66]:
dict_predict_target = {}

sen = "anger"
indexs = df_test[df_test["class"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v3.pickle", "rb"))
clf = CLF[sen]

dict_predict_target[sen] = predict_target(new_df_representation_test_v3.loc[indexs][cols_selected_sen], clf, intensities)

sen = "joy"
indexs = df_test[df_test["class"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v3.pickle", "rb"))
clf = CLF[sen]

dict_predict_target[sen] = predict_target(new_df_representation_test_v3.loc[indexs][cols_selected_sen], clf, intensities)

sen = "fear"
indexs = df_test[df_test["class"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v4.pickle", "rb"))
clf = CLF[sen]

dict_predict_target[sen] = predict_target(new_df_representation_test_v4.loc[indexs][cols_selected_sen], clf, intensities)

sen = "sadness"
indexs = df_test[df_test["class"] == sen].index
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v2.pickle", "rb"))
clf = CLF[sen]

dict_predict_target[sen] = predict_target(new_df_representation_test_v2.loc[indexs][cols_selected_sen], clf, intensities)

In [75]:
predicted_target = {}

# Crear carpeta ./predictions
if (not os.path.exists('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

# por cada target set:
for key in sentiments:
    # Predecirlo
    predicted_target[key] = dict_predict_target[key]
    
    # Guardar predicciones en archivos separados. 
    predicted_target[key].to_csv('./predictions/{}-pred.txt'.format(key),
                                 sep='\t',
                                 header=False,
                                 index=False)

# Crear archivo zip
a = shutil.make_archive('predictions', 'zip', './predictions')